<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/Processing_CrowdFlower_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization


In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import re
import string


from scipy import stats
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from sklearn.utils import shuffle 
from sklearn.model_selection import KFold

In [44]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

In [45]:
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

In [46]:
from google.colab import  drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_rows',1000)

In [51]:
from __future__ import print_function
import numpy as np
import pandas as pd
import re, sys, os, csv
from numpy import array
from numpy import asarray
from numpy import zeros
import itertools
!pip3 install emoji
import emoji
from nltk.stem.porter import PorterStemmer



#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

import pickle

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub

from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
SEED = 42   
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

import keras
import keras.utils
from keras import utils as np_utils
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras import backend as K

from keras.preprocessing import text,sequence
from keras.layers import Input
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Lambda, BatchNormalization, Concatenate
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
import matplotlib.pyplot as plt
from collections import Counter
import keras.layers as layers
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.engine import Layer
from keras.layers import TimeDistributed
from keras.layers import Activation, Flatten

sess = tf.Session()
# K.set_session(sess)



#Import Data

In [76]:
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower.csv')

df.head()

,emotion,tweets
0,empty,@tiffanylue i know i was listenin to bad habit earlier and i started freakin at his part =[
1,sadness,Layin n bed with a headache ughhhh...waitin on your call...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,"@dannycastillo We want to trade with someone who has Houston tickets, but no one will."


In [ ]:
df['emotion'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: emotion, dtype: int64

In [61]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }


In [62]:
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
	"cant":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
	"didnt":"did not",
        "doesn't":"does not",
	"doesnt":"does not",
        "don't":"do not",
	"dont":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
	"hadnt":"had not",
        "hasnt":"has not",
        "havent":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
	"Im":"I am",
	"im":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
	"mightnt":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
	"shouldnt":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
	"wasnt":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
	"werent":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
	"whos":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
	"wouldnt":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }


In [63]:
def remove_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x



In [64]:
def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)


In [65]:
def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


In [66]:
def preprocess_word(word):
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word




In [67]:


def cleanText(text): 
    processed_tweet = []
    porter_stemmer = PorterStemmer()	
    text = text.strip().replace("\n", " ").replace("\r", " ")
    #Special case not handled previously.
    text = text.replace('\x92',"'")
    #Removal of address
    text = ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())	
    #text = replace_contractions(text)
    #text = replace_links(text, "link")
    text = remove_numbers(text)
    text = re.sub(r'[,!@#$%^&*)(|/><";:.?\'\\}{]',"",text)
    text = text.lower()
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    text = text.replace("’","'")
    words = text.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    text = " ".join(reformed)
    # Standardizing words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = text.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    text = " ".join(reformed)
    #Deal with emojis
    text = emoji.demojize(text)
    #Strip accents
    text= strip_accents(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split())

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    words = text.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
             word = str(porter_stemmer.stem(word))
        processed_tweet.append(word)    
    return ' '.join(processed_tweet)
    

In [72]:
def tweet_label(label):
    if label == "empty":return 0 # neutral
    elif label == "sadness":return 2 # sad
    elif label == "enthusiasm":return 1 # happy
    elif label == "neutral":return 0 # neutral
    elif label == "worry":return 2 # sad
    elif label == "surprise":return 1 # happy
    elif label == "love":return 1 # happy
    elif label == "fun":return 1 # happy
    elif label == "hate":return 3 #hate
    elif label == "happiness":return 1 # happy
    elif label == "boredom":return 0 # neutral
    elif label == "relief":return 1 # happy
    elif label == "anger":return 3 #anger

In [77]:
df['tweets'] = df['tweets'].apply(lambda x: cleanText(x))
df['emotion'] = df['emotion'].apply(lambda x: tweet_label(x))

In [78]:
df.head()

,emotion,tweets
0,0,tiffanylu i know i wa listenin to bad habit earlier and i start freakin at hi part =[
1,2,layin n bed with a headach ughhwaitin on your call
2,2,funer ceremonygloomi friday
3,1,want to hang out with friend soon
4,0,dannycastillo we want to trade with someon who ha houston ticket but no one will


In [81]:
df.to_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/cF_clean_our_combined.csv',index=None)